# Neural and Evolutionary Leaning Project 

Group members: 

- Iris Moreira - 20240659
- Leonardo Di Caterina - 20240485
- Rafael Borges - 20240497

## Fourth Delivery - NN

In [1]:
# Standard library imports
import os
import random

# Third-party imports
import torch.nn as nn

# Slim-GSGP imports
from slim_gsgp.datasets.data_loader import load_pandas_df

In [2]:
os.chdir(os.path.join(os.getcwd(), os.pardir))
from utils.grid_search import gp_nested_cross_validation, fit_model_GridSearch, group_and_median_rmse
from utils.visualization_funcs import *
from utils.NN_utils import *
%cd notebooks/

/Users/leonardodicaterina/Documents/GitHub/Neural_Evo_Learn/Notebooks


## Load Data

In [3]:
# Reading the desired dataset
df = pd.read_csv("../data/sustavianfeed.csv", sep=';')

# Dropping the first column (index) and renaming the columns
df = df.drop(columns= ['WING TAG', 'EMPTY MUSCULAR STOMACH'])

# Moving crude protein to the end of the dataframe
df = df[[col for col in df.columns if col != 'CRUDE PROTEIN'] + ['CRUDE PROTEIN']]

In [4]:
df

,WEIGHT,HOT CARCASS WEIGHT,CARCASS WEIGHT WITH HEAD AND LEGS,COLD CARCASS WEIGHT,BREAST WEIGHT (2),THIGH WEIGHT (2),SPLEEN,LIVER,HEART,INTESTINE,GLANDULAR STOMACH,ETHER EXTRACT,CRUDE PROTEIN
0,2223.3,1429.6,1725.6,1394.0,214.0,489.4,3.716,38.636,9.305,123.171,13.170,0.38,86.105469
1,2201.9,1450.2,1769.8,1405.4,236.0,538.7,3.494,34.725,10.084,71.800,9.781,1.66,86.143472
2,2159.9,1398.4,1724.9,1461.7,241.8,512.1,4.023,31.932,10.635,61.380,6.217,0.98,86.416898
3,2198.7,1473.9,1800.4,1425.1,227.7,549.9,3.087,32.326,11.927,64.879,8.358,1.10,85.959935
4,2003.2,1291.2,1581.6,1260.1,224.7,473.2,3.723,30.105,9.855,68.562,7.572,6.34,81.693637
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2633.1,1683.2,2034.1,1637.2,213.8,610.9,4.777,45.992,12.796,74.888,7.857,1.07,88.999126
92,2346.2,1547.8,1819.5,1511.2,228.0,528.2,3.673,35.090,11.504,68.455,7.837,1.13,88.507288
93,2648.2,1722.9,2050.0,1669.8,253.1,610.0,5.176,50.505,17.194,81.502,7.332,1.91,90.375587
94,2262.6,1498.5,1813.9,1468.0,212.1,548.4,2.829,30.266,11.129,53.011,5.425,2.21,91.211353


# Nested CV with Grid Search

In [5]:
seed = 42
random.seed(seed)

# Edit the name and log directory based on the model you want to run

#MODEL_NAME = 'GP'
#MODEL_NAME = 'GSGP'
#MODEL_NAME = 'SLIM-GSGP'
MODEL_NAME = 'NN'

DATASET_NAME = MODEL_NAME +'_sustavianfeed'
LOG_DIR = './log/' + MODEL_NAME + '/'

LOG_LEVEL = 2
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

In [6]:
k_outer = 10
k_inner = 5

In [7]:
# Turning df into X and y torch.Tensors
X, y = load_pandas_df(df, X_y=True)

In [8]:
total_instances = X.shape[0]
outer_test_size = total_instances // k_outer
outer_train_size = total_instances - outer_test_size
inner_val_size = outer_train_size // k_inner
inner_train_size = outer_train_size - inner_val_size

print(f'Total Instances:\t{total_instances}\n--')
print(f'Outer Train set:\t{outer_train_size}')
print(f'Test set:\t\t{outer_test_size}\n--')
print(f'Inner Train set:\t{inner_train_size}')
print(f'Validation set:\t\t{inner_val_size}\n')

Total Instances:	96
--
Outer Train set:	87
Test set:		9
--
Inner Train set:	70
Validation set:		17



In [9]:
inner_train_size

70

## NEAT Wrapper

In [10]:
import sys
import neat

In [ ]:
# --- NEATWrapper Class (Modified to accept config_file_path) ---
class NEATWrapper:
    def __init__(self, config_file_path, X_train, y_train, X_test, y_test,
                 generations,fixed_params,
                 param_grid, seed, **other_params):

        # Convert tensors to numpy arrays for NEAT compatibility if needed
        # Assuming NEAT's activate method can handle iterables (lists/arrays)
        # If not, explicit conversion to list/numpy array might be necessary in eval_rmse/accuracy
        self.X_train = X_train.tolist() if isinstance(X_train, torch.Tensor) else X_train
        self.y_train = y_train.tolist() if isinstance(y_train, torch.Tensor) else y_train
        self.X_test = X_test.tolist() if isinstance(X_test, torch.Tensor) else X_test
        self.y_test = y_test.tolist() if isinstance(y_test, torch.Tensor) else y_test

        self.generations = generations
        self.seed = seed
        self.config_file_path = config_file_path # Store the path
        
        # I use config first so I am sure that all parameters are set correctl
        config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                           neat.DefaultSpeciesSet, neat.DefaultStagnation,
                           self.config_file_path) # Use the stored path

        # Apply param_grid to config
        for param, value in param_grid.items():
            # This assumes dynamic_params are genome_config parameters.
            # If they belong to other sections (e.g., NEAT, DefaultStagnation),
            # additional logic would be needed here.
            setattr(config.genome_config, param, value)
        
        # Apply fixed_params to config
        for param, value in fixed_params.items():
            # This assumes fixed_params are genome_config parameters.
            # If they belong to other sections (e.g., NEAT, DefaultStagnation),
            # additional logic would be needed here.
            setattr(config.genome_config, param, value)

        # Create population with seed
        p = neat.Population(config)

        # Determine if NEAT's verbose output should be redirected
        log_level = other_params.get("log_level", 0)
        neat_output_redirected = False
        if log_level > 1: # Redirect if log_level is sufficiently high (e.g., 2 for outer loop)
            timestamp = int(time.time() * 1000)
            neat_output_file = f"neat_run_stdout_{timestamp}.txt"
            print(f"Redirecting NEAT stdout to {neat_output_file}")
            with open(neat_output_file, 'w') as f:
                original_stdout = sys.stdout
                sys.stdout = f
                try:
                    self.winner = p.run(self.eval_genomes, generations)
                    neat_output_redirected = True
                finally:
                    sys.stdout = original_stdout
        else:
            # If not redirecting, still suppress some default NEAT output if needed
            # For simplicity, we just run directly if not redirecting to a file
            self.winner = p.run(self.eval_genomes, generations)


        # Calculate final fitness values for grid search compatibility
        self._calculate_final_fitness()

    def eval_rmse(self, net, X, y):
        '''
        Auxiliary function to evaluate the RMSE.
        '''
        fit = 0.
        # Ensure y is a list of single values for direct comparison if it came from torch.Tensor
        y_list = [val.item() if isinstance(val, torch.Tensor) else val for val in y]

        for xi, xo in zip(X, y_list):
            output = net.activate(xi)
            fit += (output[0] - xo)**2
        # RMSE
        return (fit/len(y_list))**.5

    def eval_accuracy(self, net, X, y):
        '''
        Auxiliary function to evaluate the accuracy.
        '''
        fit = 0.
        y_list = [val.item() if isinstance(val, torch.Tensor) else val for val in y]

        for xi, xo in zip(X, y_list):
            output = 1. if net.activate(xi)[0] >= .5 else 0.
            fit += (output==xo)
        # ACCURACY
        return fit/len(y_list)

    def eval_genomes(self, genomes, config):
        '''
        The function used by NEAT-Python to evaluate the fitness of the genomes.
        -> It has to have the two first arguments genomes and config.
        -> It has to update the `fitness` attribute of the genome.
        '''
        for genome_id, genome in genomes:
            # Define the network
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            # Train fitness (negative RMSE for maximization)
            genome.fitness = -self.eval_rmse(net, self.X_train, self.y_train)
            genome.acc = self.eval_accuracy(net, self.X_train, self.y_train)

            # Test fitness (using X_test, y_test from grid search)
            genome.fitness_val = -self.eval_rmse(net, self.X_test, self.y_test)
            genome.acc_val = self.eval_accuracy(net, self.X_test, self.y_test)

    def _calculate_final_fitness(self):
        '''
        Calculate final fitness values for grid search compatibility.
        Returns positive RMSE values as expected by the grid search framework.
        '''
        # Create network from winner
        config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                           neat.DefaultSpeciesSet, neat.DefaultStagnation,
                           self.config_file_path)  # Use the stored path

        net = neat.nn.FeedForwardNetwork.create(self.winner, config)

        # Calculate positive RMSE values (grid search expects positive values)
        self.fitness = self.eval_rmse(net, self.X_train, self.y_train)
        self.test_fitness = self.eval_rmse(net, self.X_test, self.y_test)

    def item(self):
        '''
        Compatibility method for grid search framework that expects .item() calls
        '''
        # This method is not directly used for fitness/test_fitness values themselves,
        # but ensures the object has a .item() method if called on other attributes.
        # For actual fitness values, they are directly accessed as self.fitness and self.test_fitness.
        return self

In [17]:
fixed_params = {
    # Essential for NEATWrapper initialization:
    'config_file_path': 'neat_config.txt', # Path to your NEAT config file
    'generations': 100,                  # Number of generations for each NEAT run

    # Problem-specific parameters from NEAT config:
    'num_inputs': 2,
    'num_outputs': 1,
    'fitness_criterion': 'max',
    'fitness_threshold': 0.95,

    # Core NEAT algorithm parameters (usually fixed for a given experiment):
    'feed_forward': True,
    'initial_connection': 'full_direct',
    'reset_on_extinction': False,
    'enabled_default': True,
    'enabled_mutate_rate': 0.01,
    'aggregation_default': 'sum',
    'aggregation_mutate_rate': 0.0,
    'aggregation_options': 'sum', # Only 'sum' is an option here

    # Bias mutation parameters (often fixed, but 'bias_mutate_rate' can be dynamic)
    'bias_init_mean': 0.0,
    'bias_init_stdev': 1.0,
    'bias_max_value': 30.0,
    'bias_min_value': -30.0,
    'bias_mutate_power': 0.5,
    'bias_replace_rate': 0.1,

    # Node response parameters (often fixed, but 'response_mutate_rate' can be dynamic)
    'response_init_mean': 0.0,
    'response_init_stdev': 1.0,
    'response_max_value': 30.0,
    'response_min_value': -30.0,
    'response_mutate_power': 1.0,
    'response_replace_rate': 0.0,

    # Weight mutation parameters (often fixed, but 'weight_mutate_rate' can be dynamic)
    'weight_init_mean': 0.0,
    'weight_init_stdev': 1.0,
    'weight_max_value': 30,
    'weight_min_value': -30,
    'weight_mutate_power': 0.5,
    'weight_replace_rate': 0.1,

    # Speciation and Reproduction parameters:
    'species_fitness_func': 'max',
    'max_stagnation': 20,
    'species_elitism': 2,
    'elitism': 2,
    'survival_threshold': 0.2,

    # Placeholder for data and log_level (set dynamically by nested CV function):
    'X_train': None,
    'y_train': None,
    'X_test': None,
    'y_test': None,
    'log_level': 0 # NEAT logging verbosity (0 for inner, 2 for outer)
}


param_grid = {
    # Core NEAT population & evolution
    'pop_size': [100, 200, 300], # Example: Try different population sizes

    # Node parameters
    'activation_default': ["sigmoid", "relu", "gauss"], # Example: Different default activation functions
    'activation_mutate_rate': [0.1, 0.25, 0.4], # Example: Probability of changing activation function
    'bias_mutate_rate': [0.5, 0.7, 0.9],       # Example: Probability of mutating node biases

    # Connection parameters
    'conn_add_prob': [0.6, 0.75, 0.9],  # Example: Probability of adding a new connection
    'conn_delete_prob': [0.1, 0.2, 0.3], # Example: Probability of deleting a connection
    'weight_mutate_rate': [0.7, 0.85, 1.0], # Example: Probability of mutating connection weights

    # Node addition/deletion (controls topology evolution)
    'node_add_prob': [0.05, 0.1, 0.15], # Example: Probability of adding a new node (often small)
    'node_delete_prob': [0.01, 0.05, 0.1], # Example: Probability of deleting a node (even smaller)

    # Speciation parameters
    'compatibility_disjoint_coefficient': [0.8, 1.0, 1.2], # Example: Tuning how structural differences impact speciation
    'compatibility_weight_coefficient': [0.3, 0.5, 0.7],   # Example: Tuning how weight differences impact speciation

    # Node response mutation
    'response_mutate_rate': [0.3, 0.5, 0.7], # Example: Probability of mutating node response (gain)
}

In [18]:
outer_results = gp_nested_cross_validation(X, y, gp_model=NEATWrapper, k_outer=k_outer, k_inner=k_inner, fixed_params=fixed_params, param_grid=param_grid, seed=seed, LOG_DIR=LOG_DIR, DATASET_NAME=DATASET_NAME)

Outer fold 1/10
-----
 Inner fold 1/5
Training shape: torch.Size([68, 12])
Validation shape: torch.Size([18, 12])



TypeError: NEATWrapper.__init__() missing 2 required positional arguments: 'fixed_params' and 'param_grid'

Saving results and configs to a .csv 

In [15]:
outer_results_df = pd.DataFrame(outer_results)
outer_results_df.to_csv(LOG_DIR+DATASET_NAME+'_outer_results.csv', index=False)


## Visualizations 

### Boxplot analysis

As we can see, only the 'Adam' and 'SGD' optimizers were chosen to train the outer folds. Also, it was never chosen the archictecture that adopts 3 hidden layers, with sizes (3, 4, 4).

The combination that was chosen the most times was the one with hidden_layer_sizes=(3,4), 'SGD' as the optimizer, and learning rate = 0.005.

Based on the visualizations that plots all best combinations' test rmse, we can see that the last combination presented has a much worse test rmse when compared to the others.

In [17]:
test_best_combs(model_name=MODEL_NAME)

In the boxplots comparing train and test fitness, when looking at the most chosen combinations (with 'SGD', learning rate=0.005, and 2 hidden layers) we can observe that the test values are more spread. Nonetheless, we do not have enough data points to make comparisons.

In [7]:
train_test_best_combs(model_name=MODEL_NAME)         

### Brief Discussion on Overfitting and Early Convergence

- **Good overall generalization:**  
  For most outer folds, the training and test loss curves track closely, indicating the model is generalizing well with minimal overfitting. The exception is an outer fold, where a clear gap between training and test performance suggests some overfitting (hiden_layer_sizes=(3,4), 'Adam' optimizer and learning rate=0.01)

- **Rapid convergence in some folds:**  
    Several folds exhibit a steep drop in loss during the initial epochs, giving the impression of an early plateau. However, a closer look reveals a small but consistent decrease in loss at each subsequent iteration, which ultimately drives both training and test errors to near-zero by the end of training.


In [5]:
fit_or_size_per_comb(k_outer,MODEL_NAME)

### Future Work

- For each outer, we have few data points for each combinations (chosen within the inner), with no uniformly spreadness across it, so it should be used more outer folds. This way, we could do more reliable comparisons, with more confidence on the results.

- Adicionally, in the future, we could get all fitness values from all outers and compare different algoritms by taking into account time complexity.